<a href="https://colab.research.google.com/github/jaxon-z/CSE-413-Big-Data-IoT-Lab/blob/main/CSE413_Project_Jackson%5B193_15_1076%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!gdown 1njdrzjPSQkiUcU3WHbMtxnwuynYG19ET #Download spark-3.1.1-bin-hadoop3.2.tgz
!wget https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [1,347 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [30.0 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,519 kB]
Ign:13 https://developer.download.nvidia.com/compute/

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark").getOrCreate()
spark

In [3]:
#Download Dataset: Premier League
!wget https://raw.githubusercontent.com/jaxon-z/CSE-413-Big-Data-IoT-Lab/main/Big%20Data%20Lab%20Project/11k.csv

--2022-12-05 05:44:47--  https://raw.githubusercontent.com/jaxon-z/CSE-413-Big-Data-IoT-Lab/main/Big%20Data%20Lab%20Project/11k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1082023 (1.0M) [text/plain]
Saving to: ‘11k.csv’

11k.csv             100%[===================>]   1.03M  --.-KB/s    in 0.009s  

2022-12-05 05:44:47 (109 MB/s) - ‘11k.csv’ saved [1082023/1082023]



In [ ]:
"""
FTHG	Full Time Home Team Goals
FTAG	Full Time Away Team Goals
FTR	Full Time Result (H=Home Win, D=Draw, A=Away Win)
HTHG	Half Time Home Team Goals
HTAG	Half Time Away Team Goals
HTR	Half Time Result (H=Home Win, D=Draw, A=Away Win)
Referee	Match Referee
HS	Home Team Shots
AS	Away Team Shots
HST	Home Team Shots on Target
AST	Away Team Shots on Target
HC	Home Team Corners
AC	Away Team Corners
HF	Home Team Fouls Committed
AF	Away Team Fouls Committed
HY	Home Team Yellow Cards
AY	Away Team Yellow Cards
HR	Home Team Red Cards
AR	Away Team Red Cards
"""

In [10]:
hf = spark.read.csv("/content/11k.csv", header = True, inferSchema = True)
hf.show(5)

+-------+-------------------+-----------+--------------+----+----+---+----+----+---+-------+---+---+---+---+---+---+---+---+---+---+---+---+
| Season|           DateTime|   HomeTeam|      AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|Referee| HS| AS|HST|AST| HC| AC| HF| AF| HY| AY| HR| AR|
+-------+-------------------+-----------+--------------+----+----+---+----+----+---+-------+---+---+---+---+---+---+---+---+---+---+---+---+
|1993-94|1993-08-14 00:00:00|    Arsenal|      Coventry|   0|   3|  A|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993-08-14 00:00:00|Aston Villa|           QPR|   4|   1|  H|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993-08-14 00:00:00|    Chelsea|     Blackburn|   1|   2|  A|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993-08-14 00:00:00|  Liverpool|Sheffield Weds|   2|   0|  H|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993

In [11]:
hf.printSchema()

root
 |-- Season: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTHG: integer (nullable = true)
 |-- FTAG: integer (nullable = true)
 |-- FTR: string (nullable = true)
 |-- HTHG: string (nullable = true)
 |-- HTAG: string (nullable = true)
 |-- HTR: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- HS: string (nullable = true)
 |-- AS: string (nullable = true)
 |-- HST: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- HC: string (nullable = true)
 |-- AC: string (nullable = true)
 |-- HF: string (nullable = true)
 |-- AF: string (nullable = true)
 |-- HY: string (nullable = true)
 |-- AY: string (nullable = true)
 |-- HR: string (nullable = true)
 |-- AR: string (nullable = true)



In [12]:
print(hf.count(), len(hf.columns))

11113 23


In [13]:
#df = df.drop('Name','Jersey Number','Nationality','Headed goals','Goals with right foot','Goals with left foot','Penalties scored','Freekicks scored','Shots','Shots on target','Shooting accuracy %','Hit woodwork','Big chances missed','Clean sheets','Goals conceded','Tackles','Tackle success %','Last man tackles','Blocked shots','Interceptions','Clearances','Headed Clearance','Clearances off line','Recoveries','Duels won','Duels lost','Successful 50/50s','Aerial battles won','Aerial battles lost','Own goals','Errors leading to goal','Assists','Passes','Passes per match','Big chances created','Crosses','Cross accuracy %','Through balls','Accurate long balls','Saves','Penalties saved','Punches','High Claims','Catches','Sweeper clearances','Throw outs','Goal Kicks','Yellow cards','Red cards','Fouls','Offsides')

In [14]:
df = hf.na.fill(value=0)
df.show(5)

+-------+-------------------+-----------+--------------+----+----+---+----+----+---+-------+---+---+---+---+---+---+---+---+---+---+---+---+
| Season|           DateTime|   HomeTeam|      AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|Referee| HS| AS|HST|AST| HC| AC| HF| AF| HY| AY| HR| AR|
+-------+-------------------+-----------+--------------+----+----+---+----+----+---+-------+---+---+---+---+---+---+---+---+---+---+---+---+
|1993-94|1993-08-14 00:00:00|    Arsenal|      Coventry|   0|   3|  A|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993-08-14 00:00:00|Aston Villa|           QPR|   4|   1|  H|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993-08-14 00:00:00|    Chelsea|     Blackburn|   1|   2|  A|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993-08-14 00:00:00|  Liverpool|Sheffield Weds|   2|   0|  H|  NA|  NA| NA|     NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA| NA|
|1993-94|1993

In [15]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import codecs, json
import tempfile
import requests
import base64

In [32]:
data = pd.read_csv('11k.csv', encoding='ISO-8859-1')
print(data.tail())

        Season              DateTime     HomeTeam        AwayTeam  FTHG  FTAG  \
11108  2021-22  2022-04-09T17:30:00Z  Aston Villa       Tottenham     0     4   
11109  2021-22  2022-04-10T14:00:00Z    Brentford        West Ham     2     0   
11110  2021-22  2022-04-10T14:00:00Z    Leicester  Crystal Palace     2     1   
11111  2021-22  2022-04-10T14:00:00Z      Norwich         Burnley     2     0   
11112  2021-22  2022-04-10T16:30:00Z     Man City       Liverpool     2     2   

      FTR  HTHG  HTAG HTR  ...  HST  AST   HC   AC    HF    AF   HY   AY   HR  \
11108   A   0.0   1.0   A  ...  8.0  5.0  9.0  3.0  12.0  14.0  2.0  3.0  0.0   
11109   H   0.0   0.0   D  ...  7.0  1.0  4.0  6.0   2.0   6.0  0.0  1.0  0.0   
11110   H   2.0   0.0   H  ...  3.0  3.0  3.0  4.0  11.0  12.0  1.0  1.0  0.0   
11111   H   1.0   0.0   H  ...  6.0  4.0  6.0  7.0  12.0  10.0  1.0  1.0  0.0   
11112   D   2.0   1.0   H  ...  5.0  4.0  4.0  1.0   9.0  11.0  1.0  4.0  0.0   

        AR  
11108  0.0  


In [33]:
data["HomeTeam"].unique()

array(['Arsenal', 'Aston Villa', 'Chelsea', 'Liverpool', 'Man City',
       'Newcastle', 'Oldham', 'Sheffield United', 'Southampton',
       'West Ham', 'Norwich', 'Tottenham', 'Everton', 'Ipswich', 'Leeds',
       'Wimbledon', 'Blackburn', 'Coventry', 'Man United', 'QPR',
       'Sheffield Weds', 'Swindon', 'Crystal Palace', 'Leicester',
       "Nott'm Forest", 'Bolton', 'Middlesbrough', 'Derby', 'Sunderland',
       'Barnsley', 'Charlton', 'Watford', 'Bradford', 'Fulham',
       'Birmingham', 'West Brom', 'Portsmouth', 'Wolves', 'Wigan',
       'Reading', 'Hull', 'Stoke', 'Burnley', 'Blackpool', 'Swansea',
       'Cardiff', 'Bournemouth', 'Brighton', 'Huddersfield', 'Brentford'],
      dtype=object)

In [34]:
home = input("Select home Team : ")
away = input("Select away Team : ")
print("Selected team(Home : Away) : " + home + " : " + away)

Select home Team : Arsenal
Select away Team : Chelsea
Selected team(Home : Away) : Arsenal : Chelsea


In [35]:
preResult = pd.concat([data[data['HomeTeam']==home], data[data['AwayTeam']==home]]).sort_index()
result = pd.concat([preResult[preResult['AwayTeam']==away]]).sort_index()
result

,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
405,1993-94,1994-04-16T00:00:00Z,Arsenal,Chelsea,1,0,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
560,1994-95,1994-10-15T00:00:00Z,Arsenal,Chelsea,3,1,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1093,1995-96,1995-12-16T00:00:00Z,Arsenal,Chelsea,1,1,D,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1335,1996-97,1996-09-04T00:00:00Z,Arsenal,Chelsea,3,3,D,1.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1931,1997-98,1998-02-08T00:00:00Z,Arsenal,Chelsea,2,0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2292,1998-99,1999-01-31T00:00:00Z,Arsenal,Chelsea,1,0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2803,1999-00,2000-05-06T00:00:00Z,Arsenal,Chelsea,2,1,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3039,2000-01,2001-01-13T00:00:00Z,Arsenal,Chelsea,1,1,D,1.0,0.0,H,...,8.0,8.0,10.0,3.0,13.0,15.0,1.0,4.0,0.0,0.0
3382,2001-02,2001-12-26T00:00:00Z,Arsenal,Chelsea,2,1,H,0.0,1.0,A,...,7.0,3.0,4.0,5.0,18.0,21.0,2.0,4.0,0.0,0.0
3793,2002-03,2003-01-01T00:00:00Z,Arsenal,Chelsea,3,2,H,1.0,0.0,H,...,7.0,7.0,2.0,7.0,13.0,9.0,0.0,0.0,0.0,0.0


In [36]:
new_result = result[result["Season"]>="2000"]
new_result.loc[(result.FTR == 'H'),'FTR']=0
new_result.loc[(result.FTR == 'A'),'FTR']=2
new_result.loc[(result.FTR == 'D'),'FTR']=1
x_data = new_result.iloc[:,7:9]
y_data = new_result.iloc[:,6]
y_oneHot = tf.keras.utils.to_categorical(y_data,3)
tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.Dense(units=3,input_dim=x_data.shape[1],activation='softmax'))

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [37]:
tf.model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.SGD(lr=0.1),metrics=['accuracy'])
tf.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [38]:
history = tf.model.fit(x_data,y_oneHot,epochs=500)
print("Accuracy : ",history.history['accuracy'][-1])

Epoch 1/500
1/1 [==============================] - 0s 491ms/step - loss: 0.9762 - accuracy: 0.5455
Epoch 2/500
1/1 [==============================] - 0s 12ms/step - loss: 0.9641 - accuracy: 0.5455
Epoch 3/500
1/1 [==============================] - 0s 8ms/step - loss: 0.9532 - accuracy: 0.5909
Epoch 4/500
1/1 [==============================] - 0s 10ms/step - loss: 0.9434 - accuracy: 0.5909
Epoch 5/500
1/1 [==============================] - 0s 12ms/step - loss: 0.9345 - accuracy: 0.5909
Epoch 6/500
1/1 [==============================] - 0s 18ms/step - loss: 0.9266 - accuracy: 0.5909
Epoch 7/500
1/1 [==============================] - 0s 9ms/step - loss: 0.9193 - accuracy: 0.5909
Epoch 8/500
1/1 [==============================] - 0s 7ms/step - loss: 0.9127 - accuracy: 0.5909
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 0.9067 - accuracy: 0.5909
Epoch 10/500
1/1 [==============================] - 0s 8ms/step - loss: 0.9012 - accuracy: 0.5909
Epoch 11/500
1/1 [====

In [39]:
test_hg = int(input("Enter Home Goal : "))
test_ag = int(input("Enter Away Goal : "))
test_data = np.array([[test_hg,test_ag]])
print(tf.model.predict(test_data),tf.model.predict(test_data)>0.5)

Enter Home Goal : 2
Enter Away Goal : 3
1/1 [==============================] - 0s 19ms/step
[[0.05266215 0.00152672 0.94581115]] [[False False  True]]
